KoBERT (Korean Bidirectional Encoder Representations from Transformers)는
기존 BERT의 한국어 성능 한계를 극복하기 위해 SKT Brain에서 개발한 모델이다.

이러한 KoBERT는 위키피디아나 뉴스 등에서 수집한 수백만 개 한국어 문장의 대규모 말뭉치 (Corpus)를 학습하였으며, 한국어의 불규칙한 언어 변화의 특성을 반영하기 위해 데이터 기반 토큰화 (Tokenization) 기법을 적용하여 기존 대비 27%의 토큰만으로 2.6% 이상의 성능 향상을 이끌어 낸 모델이다.

대량의 데이터를 빠른시간에 학습하기 위해 링 리듀스(ring-reduce) 기반 분산 학습 기술을 사용하여 십억 개 이상의 문장을 다수의 머신에서 빠르게 학습하고, 파이토치(PyTorch), 텐서플로우(TensorFlow), ONNX, MXNet을 포함한 다양한 딥러닝 API를 지원함으로써 많은 분야에서 언어 이해 서비스 확산에 기여하고 있다.


PyTorch >= 1.10.1 Transformers = 4.34.1 Colab batch size = 64 epochs = 5

In [ ]:
!pip install virtualenv
!virtualenv kobert_env
!source kobert_env/bin/activate

created virtual environment CPython3.10.12.final.0-64 in 205ms
  creator CPython3Posix(dest=/content/kobert_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.3.1, setuptools==68.2.2, wheel==0.41.3
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
!pip install --upgrade pip
!pip install transformers
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install torch
!pip install gluonnlp

In [ ]:
# https://github.com/SKTBrain/KoBERT 의 파일들을 Colab으로 다운로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-vvlc8dsn/kobert-tokenizer_7d9f62e2f11947f8829d56fade6ae397
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-vvlc8dsn/kobert-tokenizer_7d9f62e2f11947f8829d56fade6ae397
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel, XLNetTokenizer

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
device = torch.device("cuda:0")
tokenizer = XLNetTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

# 데이터 전처리

In [ ]:
import pandas as pd

df = pd.read_csv('/content/CrawlingSet10000_1.csv', index_col=0, encoding='cp949')

In [ ]:
df.sample(n=10)
df

,Title,Singer,Lyrics,Date,Genre
Id,,,,,
35590600,읊어본 너의 두 눈엔,그네,읊어본 너의 두 눈엔 슬픔이 가득해서 보듬어 주고픈 이 내 맘 머뭇거리는데 읊조린 ...,2022,발라드
35585693,18 (feat. Paloalto),성수민 (SEONGSOO),Baby Would you come and just dance with me Wou...,2022,R&B/Soul
35583358,빈자리,한영주,울지마 울지마 울지마 떠나간 빈자리에 앉아 마지막 순간조차 지켜주지 못했던 떠나간 ...,2022,발라드
35590685,헤어짐,김대훈,그렇게 시간이 흘러 오늘까지 와버렸네요 언제까지 그날에 갇혀 살아가야 해야하나요 한...,2022,발라드
35590378,Blended Rap,Unsigned nino (언사인드 니노),You can't mess with me. My rap is Michael Tyso...,2022,랩/힙합
...,...,...,...,...,...
36922232,THE RHYTHM OF JUSTICE,박지훈 (D.A),"We march in lockstep, Advancing into death, ou...",2023,록/메탈
36917725,도시의 별빛,문득,이른 아침 눈을 떴을 때부터 너를 그려 하루 종일 우리는 함께해 어떡하면 언제나 서...,2023,발라드
36918436,다시 누군가를 사랑한다면,한경일,니가 내게 했던 말 넌 기억나니 더 좋은 사람 만나라면서 이제 내가 싫어졌다 말을 ...,2023,발라드


In [ ]:
df=df[~df.duplicated()] #중복제거

In [ ]:
# 각 열별로 결측치의 수를 확인
print(df.isnull().sum())

Title     0
Singer    0
Lyrics    0
Date      0
Genre     0
dtype: int64


In [ ]:
# 각 열의 데이터 타입 확인
print(df.dtypes)

Title     object
Singer    object
Lyrics    object
Date       int64
Genre     object
dtype: object


In [ ]:
# 결측치를 포함하는 행을 제거 (필요하다면)
df.dropna(inplace=True)

<ipython-input-10-6645b7d6766e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [ ]:
def classify_lyrics(lyrics):
    # # 분류에 따른 키워드 정의(사랑, 이별, 우정, 가족, 꿈, 성장, 감정, 환경, 일상)
    # # keywords_1 = ["사랑", "그대", "안녕", "너무", "다시", "내가", "너를", "지금", "나를", "사랑해"]
    # keywords_1 = ["사랑", "그대", "안녕", "너무", "다시", "지금", "사랑해"]
    # # keywords_2 = ["내가", "너를", "나를", "우리", "니가", "다시", "이별", "정말", "혼자", "너무"]
    # keywords_2 = ["우리", "다시", "이별", "정말", "혼자", "너무"]
    # # keywords_3 = ["우리", "함께", "나의", "너와", "서로", "보고", "우린", "다시", "친구야", "내가"]
    # keywords_3 = ["우리", "함께", "서로", "보고", "우린", "다시", "친구야"]
    # # keywords_4 = ["엄마", "내가", "나의", "이제", "사람", "아빠", "우리", "울어", "가족", "아버지"]
    # keywords_4 = ["엄마", "이제", "사람", "아빠", "우리", "울어", "가족", "아버지"]
    # # keywords_5 = ["날아올라", "너를", "나를", "높이", "길을", "가게", "내가", "꿈을", "꿈", "멀리"]
    # keywords_5 = ["날아올라", "높이", "길을", "가게", "꿈을", "꿈", "멀리"]
    # # keywords_6 = ["내가", "거야", "나의", "너의", "함께", "나를", "다시", "이태원", "괜찮아", "성장"]
    # keywords_6 = ["거야", "함께", "다시", "이태원", "괜찮아", "성장"]
    # # keywords_7 = ["다시", "혼자", "네가", "나를", "챔피언", "그리워", "감정", "우울하다", "내가", "우울해"]
    # keywords_7 = ["다시", "혼자", "챔피언", "그리워", "감정", "우울하다", "우울해"]
    # # keywords_8 = ["좋아", "떠날래", "나의", "나도", "따라", "아침", "싶어", "환경", "커다란", "내겐"]
    # keywords_8 = ["좋아", "떠날래", "따라", "아침", "싶어", "환경", "커다란", "내겐"]
    # # keywords_9 = ["나는", "멋대로", "먹고", "죄송합니다", "일상", "달콤한", "행복을", "꿈을", "기분좋아지는", "싶은"]
    # keywords_9 = ["멋대로", "먹고", "죄송합니다", "일상", "달콤한", "행복을", "꿈을", "기분좋아지는", "싶은"]

        # 분류에 따른 키워드 정의(사랑, 이별, 우정, 가족, 꿈, 환경)
    keywords_1 = ["사랑", "그대", "안녕", "너무", "다시", "지금"]
    keywords_2 = ["우리", "다시", "이별", "정말", "혼자", "너무"]
    keywords_3 = ["우리", "함께", "서로", "보고", "다시", "친구야"]
    keywords_4 = ["엄마", "이제", "사람", "아빠", "우리", "울어", "가족", "아버지"]
    keywords_5 = ["날아", "높이", "길을", "가게", "꿈을", "꿈", "멀리"]
    keywords_6 = ["좋아", "떠날래", "따라", "아침", "싶어", "환경", "커다란", "내겐"]

    # 가사 분류
    if any(keyword in lyrics for keyword in keywords_1):
        return 1
    elif any(keyword in lyrics for keyword in keywords_2):
        return 2
    elif any(keyword in lyrics for keyword in keywords_3):
        return 3
    elif any(keyword in lyrics for keyword in keywords_4):
        return 4
    elif any(keyword in lyrics for keyword in keywords_5):
        return 5
    elif any(keyword in lyrics for keyword in keywords_6):
        return 6
    else:
        return 0  # 해당되는 분류가 없는 경우

# '가사' 칼럼의 각 행에 대해 classify_lyrics 함수를 적용하여 '분류' 칼럼을 생성
df['분류'] = df['Lyrics'].apply(classify_lyrics)

<ipython-input-11-7dd8ac9a8301>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['분류'] = df['Lyrics'].apply(classify_lyrics)


In [ ]:
df.sample(n=10)
df

,Title,Singer,Lyrics,Date,Genre,분류
Id,,,,,,
35590600,읊어본 너의 두 눈엔,그네,읊어본 너의 두 눈엔 슬픔이 가득해서 보듬어 주고픈 이 내 맘 머뭇거리는데 읊조린 ...,2022,발라드,4
35585693,18 (feat. Paloalto),성수민 (SEONGSOO),Baby Would you come and just dance with me Wou...,2022,R&B/Soul,1
35583358,빈자리,한영주,울지마 울지마 울지마 떠나간 빈자리에 앉아 마지막 순간조차 지켜주지 못했던 떠나간 ...,2022,발라드,0
35590685,헤어짐,김대훈,그렇게 시간이 흘러 오늘까지 와버렸네요 언제까지 그날에 갇혀 살아가야 해야하나요 한...,2022,발라드,2
35590378,Blended Rap,Unsigned nino (언사인드 니노),You can't mess with me. My rap is Michael Tyso...,2022,랩/힙합,1
...,...,...,...,...,...,...
36922232,THE RHYTHM OF JUSTICE,박지훈 (D.A),"We march in lockstep, Advancing into death, ou...",2023,록/메탈,0
36917725,도시의 별빛,문득,이른 아침 눈을 떴을 때부터 너를 그려 하루 종일 우리는 함께해 어떡하면 언제나 서...,2023,발라드,1
36918436,다시 누군가를 사랑한다면,한경일,니가 내게 했던 말 넌 기억나니 더 좋은 사람 만나라면서 이제 내가 싫어졌다 말을 ...,2023,발라드,1


In [ ]:
# '분류' 칼럼에서 결측값이 있는 행을 확인
df[df['분류'].isnull()]

,Title,Singer,Lyrics,Date,Genre,분류
Id,,,,,,


In [ ]:
df['분류']=df['분류'].astype(int)

<ipython-input-14-4ac6c9fc2f54>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['분류']=df['분류'].astype(int)


In [ ]:
# 예: '가사' 컬럼에서 특수문자 제거
df['Lyrics'] = df['Lyrics'].str.replace('[^\w\s]', '')

<ipython-input-15-4220a8239e23>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Lyrics'] = df['Lyrics'].str.replace('[^\w\s]', '')
<ipython-input-15-4220a8239e23>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Lyrics'] = df['Lyrics'].str.replace('[^\w\s]', '')


In [ ]:
df['가사'] = df['Lyrics']

<ipython-input-16-89d48d667272>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['가사'] = df['Lyrics']


In [ ]:
# 한글이 포함되지 않은 행 제거
df = df[df['가사'].str.contains('[가-힣]')]

In [ ]:
# '번역' 칼럼의 문자열 값들을 최대 511 글자로 제한
df['가사'] = df['가사'].str[:511]

<ipython-input-18-c7c82952ac4d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['가사'] = df['가사'].str[:511]


In [ ]:
# '번역'과 '분류' 칼럼의 값을 이용하여 새로운 리스트를 생성
data_list = [[q, str(label)] for q, label in zip(df['가사'], df['분류'])]

In [ ]:
# '' 또는 '0' 값을 가진 행 제거
df = df[~df['가사'].isin(['', '0'])]

In [ ]:
# 특정 위치의 데이터를 출력
print(data_list[0])
print(data_list[234])
print(data_list[356])
print(data_list[-1])

['읊어본 너의 두 눈엔 슬픔이 가득해서 보듬어 주고픈 이 내 맘 머뭇거리는데 읊조린 너의 입가에 두 귀를 기울이니 그리도 데인 맘 무색히 그 이름 부르는데 아아 울지 마라 그리 서럽게 울지 마라 날아가던 소쩍새 날갯짓 멈춘다 아아 어찌 그리 너를 쉼 없이 울리는가 곁이라면 내 곁이라면 웃음만 주었을 것을 훑어본 나의 마음엔 한 사람뿐이라서 외면을 위한 고갯짓은 아무 의미가 없다오 아아 울지 마라 그리 서럽게 울지 마라 날아가던 소쩍새 날갯짓 멈춘다 아아 어찌 그리 너를 쉼 없이 울리는가 곁이라면 내 곁이라면 웃음만 주었을 것을 그 모습마저 아름다운 너를 아아 스쳐가는 결코 머무르지 못하는 인연으로 남기길 바라던 맘였는데 아아 나는 울어도 좋으니 악연으로 남아도 좋으니 네 울음만 멈춰다오', '4']
['이미 끝나버린 또 반복되는 사이 아무것도 남은 건 없어 필연적인 건지 우리가 원한 건지 더는 해볼 자신이 없어 우리 그날 이후로 너무 멀어 졌단 걸 We have no tension no tension 우리 쌓은 추억이 아무 힘이 없잖아 We have no future 멀어 지려는 대로 서두르지 마 굳이 말하지는 마 그냥 멀어 지잖아 사랑했으니 우리 헤어지잔 말 굳이 말하지는 마 그럼 힘이 들잖아 먼저 말하면 후회 될까봐 오랫동안 아닌 척 했어 nowdays 시들어가는 우리 모습에 나 여기까지 다 생각 했어 이젠 어떤 말 로도 돌아 갈 수 없잖아 As you know theres no way theres no way 이제 혼자 있는게 함께 있을 때 보다 편해져 가네 In some way 멀어 지려는 대로 서두르지 마 굳이 말하지는 마 그냥 멀어 지잖아 사랑했으니 우리 헤어지잔 말 굳이 말하지는 마 그럼 힘이 들잖아 거짓말 처럼 추억은 아무 힘이 없잖아 다른 연인들처럼 지쳐가는 맘 흔들리고 있는 널 견뎌', '1']
['어떻게 지내고 있었나요 나는 요즘 영활 보며 하루를 보내요 시원하게 한번 울고 나면 그제서야 지친 내가 배가 고픈 것을 알게 돼요 시간을 갖자는

In [ ]:
# 라벨별 데이터 수 계산
label_counts = df['분류'].value_counts()

# 결과 출력
print(label_counts)

1    6761
2     824
0     421
4     284
3     278
5     271
6     167
Name: 분류, dtype: int64


# 데이터셋 분류

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=0)
print(len(dataset_train))
print(len(dataset_test))

7204
1802


데이터를 train data와 test data로 나누었다면 각 데이터가 KoBERT 모델의 입력으로 들어갈 수 있는 형태가 되도록 토큰화, 정수 인코딩, 패딩 등을 해주어야 한다

# 데이터셋 토큰화

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained('skt/kobert-base-v1')
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [ ]:
data_train[0]

(array([   2, 3437, 7268, 7126, 1544, 7327, 1955, 2959, 6559, 5940, 1407,
        5035, 3868, 1996, 1370, 6116, 2514, 7473, 4946, 1370, 7095, 2007,
        6138, 6615, 6909,  680,  401, 1469, 5032, 5561, 6484, 1469, 5400,
        6553, 5047, 5655, 6973, 2872, 3278,  680,  427,  401,  517,  403,
         427, 2925, 3155, 1469, 1539, 2859, 3320,  680,  401,  517,  425,
         427,  517, 5339, 7899, 3868, 1996, 2265, 1698,    3], dtype=int32),
 array(64, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

출력값들을 보면 3개의 array가 출력되는데, 첫 번째는 패딩된 시퀀스, 두 번째는 길이와 타입에 대한 내용, 세 번재는 어텐션 마스크 시퀀스이다. 어텐션 마스크는 BERT에 데이터가 입력되었을 때 어텐션 함수가 적용되어 연산이 된다. 이때 1로 패딩된 값들은 연산할 필요가 없기 때문에 연산을 하지 않아도 된다고 알려주는 데이터가 있어야 하는데 그게 바로 어텐션 마스크 시퀀스인 것이다. 이렇게 BERT나 KoBERT에는 어텐션 마스크 데이터도 함께 입력되어야 한다

In [ ]:
# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# BERT  모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

# optimizer와 schedule 설정
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]


optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
#KoBERT 모델 학습시키기
from tqdm.notebook import tqdm

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-31-652fc090061e>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/113 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9487718343734741 train acc 0.25
epoch 1 train acc 0.6949514503441494


<ipython-input-31-652fc090061e>:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/29 [00:00<?, ?it/s]

...

epoch 4 test acc 0.7384698275862068


  0%|          | 0/113 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.8117884397506714 train acc 0.6875
epoch 5 train acc 0.7585269174041298


  0%|          | 0/29 [00:00<?, ?it/s]

epoch 5 test acc 0.7411637931034483


새로운 문장을 테스트 할 때, 입력되는 문장을 KoBERT의 입력 형식으로 바꿔주는 코드를 작성해주어야 한다. 아래 코드를 작성하여 토큰화, 패딩, 텐서를 바꿔주고 예측을 하는 'predict' 함수를 만들어 주었다.

In [ ]:
torch.save(model.state_dict(), 'model_state_dict.pt')

In [ ]:
!nvidia-smi

Thu Nov 30 02:08:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    43W /  70W |   9265MiB / 15360MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-049c247d-e913-44dd-7192-a2f55eaf9841)


In [ ]:
tok = nlp.data.BERTSPTokenizer(tokenizer.tokenize, vocab, lower=False)

def predict(model, sentence, tokenizer, max_len, device):
    # 데이터셋 생성
    data = [[sentence, '0']]
    dataset = BERTDataset(data, 0, 1, tokenizer, vocab, max_len, True, False)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, num_workers=5)

    model.eval()

    with torch.no_grad():
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(dataloader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)

            out = model(token_ids, valid_length, segment_ids)

            test_eval = []
            for i in out:
                logits = i
                logits = logits.detach().cpu().numpy()
                test_eval.append(np.argmax(logits))

            return test_eval[0]

In [ ]:
# 임의의 가사를 입력
sentence = """
아무리 기다려도 난 못가
바보처럼 울고 있는 너의 곁에
상처만 주는 나를 왜 모르고
기다리니 떠나가란 말야
보고 싶다 보고 싶다
이런 내가 미워질만큼
울고 싶다 네게 무릎꿇고
모두 없던 일이 될 수 있다면
미칠듯 사랑했던 기억이
추억들이 너를 찾고 있지만
더 이상 사랑이란 변명에
너를 가둘수 없어
이러면 안되지만
죽을만큼 보고 싶다
보고 싶다 보고 싶다
이런 내가 미워질만큼
믿고 싶다 옳은 길이라고
너를 위해 떠나야만 한다고
미칠듯 사랑했던 기억이
추억들이 너를 찾고 있지만
더 이상 사랑이란 변명에
너를 가둘수 없어
이러면 안되지만
죽을만큼 보고 싶다
죽을만큼 잊고 싶다.
"""

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 토크나이저 로드
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

# 모델 로드
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
model.load_state_dict(torch.load('model_state_dict.pt')) # 모델 가중치 로드

# 예측 수행
prediction = predict(model, sentence, tokenizer, max_len, device)
print("예측된 클래스:", prediction)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


예측된 클래스: 1


참고 사이트:
https://velog.io/@sseq007/Kobert-%EB%AA%A8%EB%8D%B8-%EC%82%AC%EC%9A%A91
https://github.com/SKTBrain/KoBERT